# Polynomial Harmonization algorithm

## Import libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import glob
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew

import sys
sys.path.insert(0, '../') # path to our library functions

from cvasl import seperated


### get data

In [ ]:
filepath_mri = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath_mri,'StrokeMRI_pvc2c.csv') 

filepath_top = '../open_work/internal_results/cleaned_pvc2s/' 
filename_top = os.path.join(filepath_top,'TOP_pvc2c.csv') 

In [ ]:
TOP = pd.read_csv(filename_top)
StrokeMRI = pd.read_csv(filename_mri)

In [ ]:
TOP = TOP.drop(TOP.columns[0],axis=1)
#TOP
StrokeMRI = StrokeMRI.drop(StrokeMRI.columns[0],axis=1)
#StrokeMRI

In [ ]:
TOP

In [ ]:
StrokeMRI

In [ ]:
# identify polynomial columns
poly_columns = ['gm_vol', 'wm_vol', 'csf_vol',
       'gm_ivc_ratio', 'gmwm_ivc_ratio', 'wmh_vol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b', 'aca_b', 'mca_b', 'pca_b', 'totalgm_b']

In [ ]:
TOP_polys = seperated.polyfit_second_degree_to_df(
        TOP,
        'age',
        poly_columns,
)
#TOP_polys

In [ ]:
MRI_polys = seperated.polyfit_second_degree_to_df(
        StrokeMRI,
        'age',
        poly_columns,
)
MRI_polys

# Home made Harmonizartion aglorithm #1

1. take polynomials of top and mri
2. assume we want to harmonize all values as if they had been shot on the strokeMRI machine
3. take the difference betwen top point and top polynomial for each point
4. make harmonized TOp dataset by adding the differences to the stroke MRI polynomial (at the same age)
5. test the harmonized for 'ML harmony'

In [ ]:
coefficients = ['coefficient_a', 'coefficient_b', 'coefficient_c']
for column in TOP[poly_columns].columns:

    row = TOP_polys[TOP_polys['column'] == column]
    if row.empty:
        # The columns that appear "weird" below (eg. `Series([], dtype: float64)`)
        # are the columns not found, so they don't have associated coefficients..
        print('skipping', column)
        continue
    a, b, c = row[coefficients].values.flatten().tolist()
    TOP[column + '_projected'] = seperated.derived_function(TOP['age'], a, b, c)
TOP

In [ ]:
#TOP['age']

In [ ]:

coefficients = ['coefficient_a', 'coefficient_b', 'coefficient_c']
for column in TOP[poly_columns].columns:
    row = MRI_polys[MRI_polys['column'] == column]
    a_mri, b_mri, c_mri = row[coefficients].values.flatten().tolist()
    TOP[column + '_diff'] = TOP[column] - TOP[column + '_projected']
    TOP[column + '_mri_projected'] = TOP[column + '_diff'] + seperated.derived_function(TOP['age'], a_mri, b_mri, c_mri)


In [ ]:
TOP

In [ ]:
TOP_mri_homemade_harmonized = TOP[[
    'participant_id',
    'age',
    'sex',
    'gm_vol_mri_projected',
    'wm_vol_mri_projected', 
    'csf_vol_mri_projected', 
    'gm_ivc_ratio_mri_projected', 
    'gmwm_ivc_ratio_mri_projected',
    'wmh_vol_mri_projected',
    'wmh_count_mri_projected', 
    'deepwm_b_cov_mri_projected', 
    'aca_b_cov_mri_projected',
    'mca_b_cov_mri_projected',
    'pca_b_cov_mri_projected', 
    'totalgm_b_cov_mri_projected', 
    'deepwm_b_mri_projected',
    'aca_b_mri_projected',
    'mca_b_mri_projected', 
    'pca_b_mri_projected',
    'totalgm_b_mri_projected',]]
len(TOP_mri_homemade_harmonized )

In [ ]:
TOP_mri_homemade_harmonized =TOP_mri_homemade_harmonized.rename(columns={
    'gm_vol_mri_projected':'gm_vol',
    'wm_vol_mri_projected':'wm_vol', 
    'csf_vol_mri_projected':'csf_vol',
    'gm_ivc_ratio_mri_projected':'gm_ivc_ratio', 
    'gmwm_ivc_ratio_mri_projected':'gmwm_ivc_ratio',
    'wmh_vol_mri_projected':'wmh_vol',
    'wmh_count_mri_projected':'wmh_count', 
    'deepwm_b_cov_mri_projected': 'deepwm_b_cov', 
    'aca_b_cov_mri_projected':'aca_b_cov',
    'mca_b_cov_mri_projected':'mca_b_cov',
    'pca_b_cov_mri_projected':'pca_b_cov', 
    'totalgm_b_cov_mri_projected':'totalgm_b_cov', 
    'deepwm_b_mri_projected':'deepwm_b',
    'aca_b_mri_projected':'aca_b',
    'mca_b_mri_projected':'mca_b', 
    'pca_b_mri_projected':'pca_b',
    'totalgm_b_mri_projected':'totalgm_b',
})
len(TOP_mri_homemade_harmonized )

In [ ]:
TOP_mri_homemade_harmonized 

In [ ]:
TOP_mri_homemade_harmonized.to_csv('TOP_mri_homemade_harmonized.csv') 